# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/859c9ca9ede14fdea841be627c412322)
  - Slide: [link](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW04/HW04.pdf)
  - Data: [link](https://drive.google.com/file/d/1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe/view?usp=sharing)
  - Video (Chinese): [link](https://www.youtube.com/watch?v=EPerg2UnGaI)
  - Video (English): [link](https://www.youtube.com/watch?v=Gpz6AUvCak0)
  - Solution for downloading dataset fail.: [link](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing)

# Download dataset
- Please follow [here](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing) to download data
- Data is [here](https://drive.google.com/file/d/1gaFy8RaQVUEXo2n0peCBR5gYKCB-mNHc/view?usp=sharing)

In [247]:
#!gdown --id 'paste your own data download link' --output Dataset.zip
#!unzip Dataset.zip

# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [248]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from time import sleep
 
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len
 
    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
 
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]
 
    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
  def __len__(self):
    return len(self.data)
 
  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))
 
    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker
 
  def get_speaker_number(self):
    return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [249]:
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=False,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=False,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [250]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.models


    
class Classifier(nn.Module):
  def __init__(self, d_model=120, n_spks=600, dropout=0.2,device=torch.device("cuda")):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=4
    )
    self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
    
    self.conformerblock=torchaudio.models.Conformer(
        input_dim=d_model,
        num_heads=4,
        ffn_dim=256,
        num_layers=2,
        depthwise_conv_kernel_size=31,
        dropout=dropout)
    
    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, 256),
      nn.ReLU(),
      nn.Linear(256, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    #out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    lengths=out.shape[1]
    batchsize=out.shape[0]
    out_len=torch.ones(batchsize)*lengths
    out_len = out_len.to(device)
    out,_ = self.conformerblock(out,out_len)
    # out: (batch size, length, d_model)
    #out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [251]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [252]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [253]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )
    sleep(0.01)

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


# Main function

In [254]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 0,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")
  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  print("get loader finished")
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num,device=device).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )
    sleep(0.01)
    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == '__main__':
  torch.set_num_threads(4)
  main(**parse_args())


[Info]: Use cuda now!
get loader finished
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [02:18<00:00, 14.43 step/s, accuracy=0.34, loss=3.27, step=2000]
Valid: 100% 6944/6944 [00:14<00:00, 491.89 uttr/s, accuracy=0.26, loss=3.50]
Train: 100% 2000/2000 [02:20<00:00, 14.22 step/s, accuracy=0.41, loss=2.97, step=4000]
Valid: 100% 6944/6944 [00:14<00:00, 475.89 uttr/s, accuracy=0.40, loss=2.74]
Train: 100% 2000/2000 [02:20<00:00, 14.21 step/s, accuracy=0.59, loss=1.65, step=6000]
Valid: 100% 6944/6944 [00:14<00:00, 493.95 uttr/s, accuracy=0.47, loss=2.31]
Train: 100% 2000/2000 [02:20<00:00, 14.22 step/s, accuracy=0.69, loss=1.91, step=8000]
Valid: 100% 6944/6944 [00:14<00:00, 477.75 uttr/s, accuracy=0.54, loss=2.01]
Train: 100% 2000/2000 [02:20<00:00, 14.24 step/s, accuracy=0.59, loss=1.55, step=1e+4]
Valid: 100% 6944/6944 [00:14<00:00, 475.16 uttr/s, accuracy=0.57, loss=1.89]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                    

Train:   0% 3/2000 [00:00<02:11, 15.16 step/s, accurac

Step 10000, best model saved. (accuracy=0.5698)


Train: 100% 2000/2000 [02:19<00:00, 14.29 step/s, accuracy=0.56, loss=1.24, step=12000]
Valid: 100% 6944/6944 [00:14<00:00, 494.34 uttr/s, accuracy=0.60, loss=1.72]
Train: 100% 2000/2000 [02:23<00:00, 13.91 step/s, accuracy=0.75, loss=1.06, step=14000]
Valid: 100% 6944/6944 [00:13<00:00, 511.22 uttr/s, accuracy=0.63, loss=1.58]
Train: 100% 2000/2000 [02:13<00:00, 14.98 step/s, accuracy=0.66, loss=1.39, step=16000]
Valid: 100% 6944/6944 [00:14<00:00, 488.06 uttr/s, accuracy=0.66, loss=1.48]
Train: 100% 2000/2000 [02:17<00:00, 14.53 step/s, accuracy=0.84, loss=0.85, step=18000]
Valid: 100% 6944/6944 [00:14<00:00, 486.94 uttr/s, accuracy=0.67, loss=1.44]
Train: 100% 2000/2000 [02:18<00:00, 14.46 step/s, accuracy=0.84, loss=0.64, step=2e+4] 
Valid: 100% 6944/6944 [00:14<00:00, 479.07 uttr/s, accuracy=0.70, loss=1.32]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                    

Train:   0% 3/2000 [00:00<01:59, 16.68 step/s, ac

Step 20000, best model saved. (accuracy=0.7013)


...

Step 50000, best model saved. (accuracy=0.8167)


Train: 100% 2000/2000 [02:12<00:00, 15.06 step/s, accuracy=0.91, loss=0.26, step=52000]
Valid: 100% 6944/6944 [00:14<00:00, 486.04 uttr/s, accuracy=0.83, loss=0.80]
Train: 100% 2000/2000 [02:12<00:00, 15.04 step/s, accuracy=0.91, loss=0.41, step=54000]
Valid: 100% 6944/6944 [00:14<00:00, 489.53 uttr/s, accuracy=0.83, loss=0.77]
Train: 100% 2000/2000 [02:11<00:00, 15.16 step/s, accuracy=0.94, loss=0.19, step=56000]
Valid: 100% 6944/6944 [00:13<00:00, 498.27 uttr/s, accuracy=0.83, loss=0.77]
Train: 100% 2000/2000 [02:12<00:00, 15.14 step/s, accuracy=0.88, loss=0.50, step=58000]
Valid: 100% 6944/6944 [00:14<00:00, 483.00 uttr/s, accuracy=0.83, loss=0.78]
Train: 100% 2000/2000 [02:11<00:00, 15.22 step/s, accuracy=0.94, loss=0.25, step=6e+4] 
Valid: 100% 6944/6944 [00:14<00:00, 493.15 uttr/s, accuracy=0.83, loss=0.77]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                    

Train:   0% 3/2000 [00:00<02:23, 13.95 step/s, ac

Step 60000, best model saved. (accuracy=0.8317)


Train: 100% 2000/2000 [02:12<00:00, 15.12 step/s, accuracy=0.94, loss=0.23, step=62000]
Valid: 100% 6944/6944 [00:14<00:00, 486.92 uttr/s, accuracy=0.84, loss=0.75]
Train: 100% 2000/2000 [02:12<00:00, 15.14 step/s, accuracy=0.91, loss=0.43, step=64000]
Valid: 100% 6944/6944 [00:13<00:00, 497.96 uttr/s, accuracy=0.84, loss=0.76]
Train: 100% 2000/2000 [02:12<00:00, 15.12 step/s, accuracy=0.94, loss=0.29, step=66000]
Valid: 100% 6944/6944 [00:14<00:00, 495.16 uttr/s, accuracy=0.84, loss=0.75]
Train: 100% 2000/2000 [02:13<00:00, 15.02 step/s, accuracy=0.94, loss=0.24, step=68000]
Valid: 100% 6944/6944 [00:14<00:00, 486.41 uttr/s, accuracy=0.84, loss=0.73]
Train: 100% 2000/2000 [02:11<00:00, 15.17 step/s, accuracy=0.91, loss=0.62, step=7e+4] 
Valid: 100% 6944/6944 [00:14<00:00, 495.06 uttr/s, accuracy=0.84, loss=0.76]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                    

Train:   0% 0/2000 [00:00<?, ? step/s].09 step/s,

Step 70000, best model saved. (accuracy=0.8432)


# Inference

## Dataset of inference

In [255]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [256]:
import json
import csv
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "Dataset",
    "model_path": "model.ckpt",
    "output_path": "output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == '__main__':
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


100%|█████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:32<00:00, 186.26it/s]
